In [1]:
# http://www.hankcs.com/nlp/word2vec.html
# https://zhuanlan.zhihu.com/p/53425736 cbow 多个词加权后在做输出

In [2]:
def get_window_sample(input_list, window_size):
    left_context = []
    right_context = []
    target = []
    for idx, item in enumerate(input_list):
        if idx+window_size+1 < len(input_list):
#             print(input_list[idx:idx+window_size])
            temp_left = input_list[idx:idx+window_size]
            temp_right = input_list[idx+window_size+1:idx+2*window_size+1]
            left_context.append(temp_left)
            right_context.append(temp_right)
            target.append(input_list[idx+window_size])
    return left_context, right_context, target

In [3]:
def preprocessing(char_list):
    from numpy import array
    from numpy import argmax
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(array(char_list))
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)   
    return integer_encoded, onehot_encoded

In [4]:
def CBOW(self):
    self.context = tf.placeholder(dtype=tf.float32, shape=[None, windows_size, vocab_size])
    self.left = tf.placeholder(dtype=tf.float32, shape=[None, windows_size, vocab_size])
    self.target = tf.placeholder(tf.float32, shape=[None, vocab_size])

    w_1 = tf.Variable(tf.random_uniform([vocab_size, embedding_dim], stddev=0.1))
    w_2 = tf.Variable(tf.random_uniform([embedding_dim, vocab_size], stddev=0.1))
                      
    left_encode = tf.matmul(left, w1)
    left_sum = tf.reduce_sum(left_encode, axis=1)
                      
    right_encode = tf.matmul(right, w1)
    right_sum = tf.reduce_sum(right_encode, axis=1)
    
    context_encode = tf.concat([left_encode, right_encode], 0)
    context_sum = tf.reduce_sum(context_encode, axis=1)
    
    outputs = tf.matmul(context_sum, w_2)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=target)
    loss = tf.reduce_mean(loss)
    
    self.trainer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)


In [5]:
import tensorflow as tf
import numpy as np

text = "火星（Mars）是太阳系八大行星之一，是太阳系由内往外数的第四颗行星，属于类地行星，直径约为地球的53%，质量为地球的14%。自转轴倾角、自转周期均与地球相近，公转一周约为地球公转时间的两倍。橘红色外表是地表的赤铁矿（氧化铁）。我国古书上将火星称为“荧惑星”，西方古代（古罗马）称为“神话玛尔斯星”是十二星座白羊座的独一守护星，并非天蝎座的守护行星。火星基本上是沙漠行星，地表沙丘、砾石遍布且没有稳定的液态水体（2015年9月28日，美国宇航局公布火星上有少量的水。据法新社2018年7月25日报道，欧洲航天局(ESA)的研究员称，火星上发现了第一个液态地下水湖）。二氧化碳为主的大气既稀薄又寒冷，沙尘悬浮其中，每年常有尘暴发生。火星两极皆有水冰与干冰组成的极冠会随着季节消长。与地球相比，火星地质活动较不活跃，地表地貌大部分于远古较活跃的时期形成，有密布的陨石坑、火山与峡谷，包括太阳系最高的山：奥林帕斯山和最大的峡谷：水手号峡谷。另一个地形特征是南北半球的明显差别：南方是古老、充满陨石坑的高地，北方则是较年轻的平原。火星有两个天然卫星：火卫一和火卫二，形状不规则，可能是被隔离的矮小行星。在地球，火星肉眼可见，最高亮度可达-2.9等，八大行星中比木星、金星暗。2015年9月28日，美国航天局宣布火星存在流动水"
char_list = list(text)

char_num = len(set(char_list))
embedding_dim = 50
windows_size = 5
vocab_size = char_num

integer_encoded, onehot_encoded = preprocessing(char_list)
print("integer encode", integer_encoded[:3])
print("onehot_encoded", onehot_encoded[:3])
left_context, right_context, target = get_window_sample(onehot_encoded, 4)

print("left_encode", left_context[:3])
print("right_context", right_context[:3])
print("target", target[:3])

print(type(left_context))
print(type(right_context))
print(type(target))

print(tf.shape(np.asarray(left_context)))
print(tf.shape(np.asarray(right_context)))
# print(tf.shape(np.asarray(target)))

# data_loader = tf.data.Dataset.from_tensor_slices(
#     {'left':np.asarray(left_context), 
#      'right':np.asarray(right_context),
#      'target':np.asarray(target)})  

# # run demo
# iterator = data_loader.make_one_shot_iterator()
# one_element = iterator.get_next()

# sess = tf.Session()
# model = CBOW()

# while True:
#     try:
#         sample_batched = sess.run(one_element)
#         left = sample_batched['left']
#         right = sample_batched['right']
#         target = sample_batched['target']
        
#         outputs = sess.run(model.trainer,
#                            feed_dict = {model.left:left,
#                                         model.right:right,
#                                         model.target:target})
#     except tf.errors.OutOfRangeError:
#         break

integer encode [[170]
 [138]
 [256]]
onehot_encoded [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
